In [3]:
#pip install pandas

In [4]:
import pandas, json
from datetime import date

#function for getting data from the woolworths api
def download(url = 'http://www.wikipedia.org/',
             target_filename = 'download',
             filename_extension = 'html',
             save_file = True,
             char_set = 'UTF-8',
             lying = False,
             got_the_message = True):

    # Import the function for opening online documents and
    # the class for creating requests
    from urllib.request import urlopen, Request

    # Import an exception raised when a web server denies access
    # to a document
    from urllib.error import HTTPError

    # Open the web document for reading
    try:
        if lying:
            # Pretend to be something other than a Python
            # script (NOT RELIABLE OR RECOMMENDED!)
            request = Request(url)
            request.add_header('User-Agent', 'Mozilla/5.0')
            if not got_the_message:
                print("Warning - Request does not reveal client's true identity.")
                print("          This is both unreliable and unethical!")
                print("          Proceed at your own risk!\n")
        else:
            # Behave ethically
            request = url
        web_page = urlopen(request)
    except ValueError:
        print("Download error - Cannot find document at URL '" + url + "'\n")
        return None
    except HTTPError:
        print("Download error - Access denied to document at URL '" + url + "'\n")
        return None
    except Exception as message:
        print("Download error - Something went wrong when trying to download " + \
              "the document at URL '" + url + "'")
        print("Error message was:", message, "\n")
        return None

    # Read the contents as a character string
    try:
        web_page_contents = web_page.read().decode(char_set)
    except UnicodeDecodeError:
        print("Download error - Unable to decode document from URL '" + \
              url + "' as '" + char_set + "' characters\n")
        return None
    except Exception as message:
        print("Download error - Something went wrong when trying to decode " + \
              "the document from URL '" + url + "'")
        print("Error message was:", message, "\n")
        return None

    # Optionally write the contents to a local text file
    # (overwriting the file if it already exists!)
    if save_file:
        try:
            text_file = open(target_filename + '.' + filename_extension,
                             'w', encoding = char_set)
            text_file.write(web_page_contents)
            text_file.close()
        except Exception as message:
            print("Download error - Unable to write to file '" + \
                  target_filename + "'")
            print("Error message was:", message, "\n")

    # Return the downloaded document to the caller
    return web_page_contents

In [43]:
#reading in dataframe with previous prices
df = pandas.read_csv('out.csv', parse_dates=['Date'], index_col=0)
df

,Date,Name,Price
0,2023-02-09,Fresh Strawberry 250G,3.00
0,2023-02-09,White Seedless Grapes Bunch Each,6.56
0,2023-02-09,Fresh Blueberries 125G,3.50
0,2023-02-09,Driscoll's Fresh Raspberry 125G,2.90


In [40]:
#TODO: ADD ALL PRODUCTS YOU WANT TO WATCH. To get the URL, navigate to the item's webpage, open DevTools and then open Network and activate Fetch/XHR
#items = ['https://www.woolworths.com.au/api/v3/ui/schemaorg/product/144607', 'https://www.woolworths.com.au/api/v3/ui/schemaorg/product/138801', 'https://www.woolworths.com.au/api/v3/ui/schemaorg/product/169792', 'https://www.woolworths.com.au/api/v3/ui/schemaorg/product/165262']
items = []

#getting all the current prices from today
for i in items:
    data = json.loads(download(i, lying=True, save_file=False))
    df = pandas.concat([df, pandas.DataFrame([[date.today(), data['name'], data['offers']['price']]],
                   columns=['Date', 'Name', 'Price'])])

df

,Date,Name,Price
0,2023-02-09 00:00:00,Fresh Strawberry 250G,3.00
0,2023-02-09 00:00:00,White Seedless Grapes Bunch Each,6.56
0,2023-02-09 00:00:00,Fresh Blueberries 125G,3.50
0,2023-02-09,Driscoll's Fresh Raspberry 125G,2.90


In [41]:
#saves the current dataframe and reloads it so that the dates are parsed
df.to_csv('out.csv')
df = pandas.read_csv('out.csv', parse_dates=['Date'], index_col=0)

In [42]:
df

,Date,Name,Price
0,2023-02-09,Fresh Strawberry 250G,3.00
0,2023-02-09,White Seedless Grapes Bunch Each,6.56
0,2023-02-09,Fresh Blueberries 125G,3.50
0,2023-02-09,Driscoll's Fresh Raspberry 125G,2.90


In [ ]:
#TODO: DO CALCULATIONS, ADD CHARTS ETC.